In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.utils import get_last_market_day
import pandas_market_calendars as mcal
from src.constants import strftime
from src.utils import calculate_atr
from datetime import datetime, timedelta

In [12]:
def get_market_hours(date:str, exchange:str='NYSE') -> tuple:
    exch = mcal.get_calendar(exchange)
    schedule = exch.schedule(start_date=date, end_date=date)
    market_open = schedule.at[date, 'market_open']
    market_close = schedule.at[date, 'market_close']
    return market_open.time(), market_close.time()

In [2]:
def get_symbol(df, symbol:str) -> pd.DataFrame:
    return df[symbol].copy()

def reaggregate_bars(df, resample:str='1d') -> pd.DataFrame:
    return pd.DataFrame(
    {
        'Open':df['Open'].resample(resample).first(),
        'High':df['High'].resample(resample).max(),
        'Low':df['Low'].resample(resample).min(),
        'Close':df['Close'].resample(resample).last(),
        'Volume':df['Volume'].resample(resample).sum(),
    }
    ).dropna()

In [3]:
def calculate_tpo(
        df_symbol:pd.DataFrame, 
        session_date:str=get_last_market_day(),
        atr_scaler:float=25.0
    ) -> pd.Series:

    df_daily = reaggregate_bars(df_symbol, '1d')
    df = df_symbol.loc[session_date].copy()
    max_price = df['High'].max().round(2)
    min_price = df['Low'].min().round(2)
    atr = calculate_atr(df_daily).iloc[-1]
    tpo_bin_size = round(atr/atr_scaler, 2)
    bin_edges = np.arange(min_price, max_price + tpo_bin_size, tpo_bin_size)
    bin_labels = bin_edges[:-1]
    tpo_counts = pd.Series(0, index=bin_labels)
    for _, row in df.iterrows():
        low = row['Low']
        high = row['High']
        price_range = np.arange(low, high + tpo_bin_size, tpo_bin_size)
        binned = pd.cut(price_range, bins=bin_edges, labels=bin_labels, right=False)
        binned = binned.dropna().unique()
        for bin_label in binned:
            tpo_counts[bin_label] += 1
    return tpo_counts

In [49]:
def calculate_value_area(tpo_counts:pd.Series, value_area_pct:float=0.7) -> dict:
    sorted_tpo = tpo_counts.sort_values(ascending=False)
    total_tpos = sorted_tpo.sum()
    target_tpos = total_tpos * value_area_pct
    max_count = sorted_tpo.iloc[0]
    poc_candidates = sorted_tpo[sorted_tpo == max_count].index
    price_center = np.average(tpo_counts.index, weights=tpo_counts.values)
    poc = min(poc_candidates, key=lambda x: abs(x - price_center))  # closest to center
    value_area_bins = [poc]
    cumulative = tpo_counts[poc]
    remaining = tpo_counts.drop(index=poc) # remove POC bin
    sorted_bins = sorted(remaining.index)
    bin_size = sorted_bins[1] - sorted_bins[0]
    above = poc + bin_size
    below = poc - bin_size

    while cumulative < target_tpos:
        next_bins = []
        if above in remaining:
            next_bins.append((above, tpo_counts[above]))
        if below in remaining:
            next_bins.append((below, tpo_counts[below]))

        if not next_bins:
            break

    # Choose the bin with higher TPO count; if tied, prefer lower price
        next_bins.sort(key=lambda x: (-x[1], x[0]))
        chosen_price, count = next_bins[0]

        value_area_bins.append(chosen_price)
        cumulative += count
        remaining = remaining.drop(index=chosen_price)

        # Move pointers
        if chosen_price == above:
            above += bin_size
        else:
            below -= bin_size

    val = min(value_area_bins)
    vah = max(value_area_bins)

    return {
        'POC':round(poc,2),
        'VAH':round(vah,2),
        'VAL':round(val,2),
        # 'Value Area Bins': sorted(value_area_bins),
        # 'Total TPOs': total_tpos,
        # 'Captured TPOs': cumulative
    }

In [5]:
with open('files/watchlist.txt') as f:
    watchlist = f.read()

In [6]:
df_raw = yf.download(
    watchlist.split(' '), 
    auto_adjust=False, 
    interval='30m', 
    period='1mo', 
    group_by='ticker'
)

[*********************100%***********************]  80 of 80 completed


In [32]:
last_market_day = get_last_market_day()
previous_market_day = (datetime.strptime(last_market_day, strftime).date() - timedelta(days=1)).strftime(strftime)
opening_bell, closing_bell = get_market_hours(last_market_day)
first_hour = [opening_bell, (datetime.combine(datetime.today(),opening_bell) + timedelta(minutes=30)).time()]
last_half_hour = [(datetime.combine(datetime.today(),closing_bell) - timedelta(minutes=30)).time()]

In [125]:
symbol = 'AMZN'
df_symbol = get_symbol(df_raw, symbol)
df_symbol['First Hour'] = df_symbol.index.map(lambda x: x.time()).isin(first_hour)
df_symbol['Last Half Hour'] = df_symbol.index.map(lambda x: x.time()).isin(last_half_hour)

In [126]:
available_dates = list(set(df_symbol.index.date))
value_areas = dict()
for date in available_dates:
    current_date = date.strftime(strftime)
    tpo = calculate_tpo(df_symbol, current_date)
    value_areas[current_date] = calculate_value_area(tpo)
    previous_date = (date - timedelta(days=1)).strftime(strftime)

In [128]:
df_date = df_symbol.loc[current_date].copy()
va_date = value_areas[current_date]
va_prev = value_areas[previous_date]
session_low = df_date['Low'].min()
session_high = df_date['High'].max()
session_close = df_date[df_date['Last Half Hour']].iloc[-1]['Close']

ib_low = df_date[df_date['First Hour']]['Low'].min()
ib_high = df_date[df_date['First Hour']]['High'].max()

rank_extreme = 0.0
if session_low >= ib_low:
    if session_low > va_prev['VAH']:
        rank_extreme = 4.0
    elif (session_low <= va_prev['VAH']) & (session_low > va_prev['POC']):
        rank_extreme = 3.0
    elif (session_low <= va_prev['POC']) & (session_low >= va_prev['VAL']):
        rank_extreme = 2.0
    elif session_low < va_prev['VAL']:
        rank_extreme = 1.0

rank_extension = 0.0
if session_high >= ib_high:
    if session_high > va_prev['VAH']:
        rank_extension = 4.0
    elif (session_high <= va_prev['VAH']) & (session_high > va_prev['POC']):
        rank_extension = 3.0
    elif (session_high <= va_prev['POC']) & (session_high >= va_prev['VAL']):
        rank_extension = 2.0
    elif session_high < va_prev['VAL']:
        rank_extension = 1.0
        
rank_close = 0.0
if session_close > va_date['VAH']:
    rank_close = 3.0
elif session_close == va_date['VAH']:
    rank_close = 2.0
elif (session_close < va_date['VAH']) & (session_close > va_date['POC']):
    rank_close = 1.0
elif session_close == va_date['POC']:
    rank_close = 0.0
elif (session_close < va_date['POC']) & (session_close > va_date['VAL']):
    rank_close = -1.0
elif session_close == va_date['VAL']:
    rank_close = -2.0
elif session_close < va_date['VAL']:
    rank_close = -3.0

print(current_date)
print(rank_extreme, rank_extension, rank_close)
print(rank_close + rank_extension + rank_extreme)

2025-07-18
0.0 4.0 3.0
7.0


In [154]:
results = dict()

for symbol in watchlist.split(' '):
    df_symbol = get_symbol(df_raw, symbol)
    df_symbol['First Hour'] = df_symbol.index.map(lambda x: x.time()).isin(first_hour)
    df_symbol['Last Half Hour'] = df_symbol.index.map(lambda x: x.time()).isin(last_half_hour)
    available_dates = list(set(df_symbol.index.date))
    value_areas = dict()
    results[symbol] = dict()
    for date in available_dates:
        current_date = date.strftime(strftime)
        tpo = calculate_tpo(df_symbol, current_date)
        value_areas[current_date] = calculate_value_area(tpo)
        results[symbol]['Value Areas'] = value_areas

    for date in available_dates:
        previous_date = (date - timedelta(days=1)).strftime(strftime)
        if pd.to_datetime(previous_date).date() in available_dates:
            df_date = df_symbol.loc[current_date].copy()
            va_date = value_areas[current_date]
            va_prev = value_areas[previous_date]
            session_low = df_date['Low'].min()
            session_high = df_date['High'].max()
            session_close = df_date[df_date['Last Half Hour']].iloc[-1]['Close']

            ib_low = df_date[df_date['First Hour']]['Low'].min()
            ib_high = df_date[df_date['First Hour']]['High'].max()

            rank_extreme = 0.0
            if session_low >= ib_low:
                if session_low > va_prev['VAH']:
                    rank_extreme = 4.0
                elif (session_low <= va_prev['VAH']) & (session_low > va_prev['POC']):
                    rank_extreme = 3.0
                elif (session_low <= va_prev['POC']) & (session_low >= va_prev['VAL']):
                    rank_extreme = 2.0
                elif session_low < va_prev['VAL']:
                    rank_extreme = 1.0

            rank_extension = 0.0
            if session_high >= ib_high:
                if session_high > va_prev['VAH']:
                    rank_extension = 4.0
                elif (session_high <= va_prev['VAH']) & (session_high > va_prev['POC']):
                    rank_extension = 3.0
                elif (session_high <= va_prev['POC']) & (session_high >= va_prev['VAL']):
                    rank_extension = 2.0
                elif session_high < va_prev['VAL']:
                    rank_extension = 1.0
                    
            rank_close = 0.0
            if session_close > va_date['VAH']:
                rank_close = 3.0
            elif session_close == va_date['VAH']:
                rank_close = 2.0
            elif (session_close < va_date['VAH']) & (session_close > va_date['POC']):
                rank_close = 1.0
            elif session_close == va_date['POC']:
                rank_close = 0.0
            elif (session_close < va_date['POC']) & (session_close > va_date['VAL']):
                rank_close = -1.0
            elif session_close == va_date['VAL']:
                rank_close = -2.0
            elif session_close < va_date['VAL']:
                rank_close = -3.0
            results[symbol][current_date] = {'extreme':rank_extreme, 'extension':rank_extension, 'close':rank_close}

In [157]:
results['AAPL']

{'Value Areas': {'2025-06-27': {'POC': 201.42, 'VAH': 202.17, 'VAL': 200.82},
  '2025-07-07': {'POC': 209.85, 'VAH': 212.25, 'VAL': 208.8},
  '2025-07-22': {'POC': 213.58, 'VAH': 214.03, 'VAL': 212.98},
  '2025-07-08': {'POC': 209.65, 'VAH': 210.55, 'VAL': 209.35},
  '2025-07-11': {'POC': 211.06, 'VAH': 211.36, 'VAL': 210.61},
  '2025-07-24': {'POC': 214.58, 'VAH': 215.03, 'VAL': 214.13},
  '2025-07-10': {'POC': 212.37, 'VAH': 213.12, 'VAL': 211.92},
  '2025-07-21': {'POC': 213.29, 'VAH': 214.04, 'VAL': 212.39},
  '2025-07-01': {'POC': 208.09, 'VAH': 208.69, 'VAL': 207.04},
  '2025-07-14': {'POC': 208.89, 'VAH': 209.34, 'VAL': 208.29},
  '2025-07-25': {'POC': 214.3, 'VAH': 214.6, 'VAL': 213.85},
  '2025-06-26': {'POC': 200.21, 'VAH': 200.96, 'VAL': 199.61},
  '2025-07-23': {'POC': 214.21, 'VAH': 214.66, 'VAL': 213.01},
  '2025-06-30': {'POC': 200.31, 'VAH': 203.31, 'VAL': 199.26},
  '2025-07-09': {'POC': 209.62, 'VAH': 210.52, 'VAL': 207.97},
  '2025-07-15': {'POC': 210.57, 'VAH': 211.

In [155]:
pd.DataFrame(results)

,AAPL,AMD,AMZN,AVGO,C,COST,CVX,DIA,GLD,GOOG,...,UNH,UPS,WFC,XHB,XLI,XLK,XLV,XOM,XOP,XYZ
Value Areas,"{'2025-06-27': {'POC': 201.42, 'VAH': 202.17, ...","{'2025-06-27': {'POC': 143.74, 'VAH': 144.79, ...","{'2025-06-27': {'POC': 219.74, 'VAH': 221.09, ...","{'2025-06-27': {'POC': 269.58, 'VAH': 270.78, ...","{'2025-06-27': {'POC': 84.3, 'VAH': 84.44, 'VA...","{'2025-06-27': {'POC': 987.95, 'VAH': 989.99, ...","{'2025-06-27': {'POC': 143.41, 'VAH': 143.77, ...","{'2025-06-27': {'POC': 437.2, 'VAH': 438.25, '...","{'2025-06-27': {'POC': 301.57, 'VAH': 301.99, ...","{'2025-06-27': {'POC': 174.42, 'VAH': 175.44, ...",...,"{'2025-06-27': {'POC': 307.48, 'VAH': 309.46, ...","{'2025-06-27': {'POC': 101.47, 'VAH': 101.63, ...","{'2025-06-27': {'POC': 79.87, 'VAH': 80.08, 'V...","{'2025-06-27': {'POC': 98.57, 'VAH': 99.07, 'V...","{'2025-06-27': {'POC': 146.57, 'VAH': 147.11, ...","{'2025-06-27': {'POC': 252.03, 'VAH': 252.63, ...","{'2025-06-27': {'POC': 134.61, 'VAH': 134.96, ...","{'2025-06-27': {'POC': 109.01, 'VAH': 109.33, ...","{'2025-06-27': {'POC': 126.83, 'VAH': 127.16, ...","{'2025-06-27': {'POC': 66.28, 'VAH': 66.78, 'V..."
2025-07-18,"{'extreme': 1.0, 'extension': 4.0, 'close': 1.0}","{'extreme': 0.0, 'extension': 4.0, 'close': -1.0}","{'extreme': 0.0, 'extension': 4.0, 'close': 3.0}","{'extreme': 0.0, 'extension': 3.0, 'close': 1.0}","{'extreme': 3.0, 'extension': 4.0, 'close': 1.0}","{'extreme': 0.0, 'extension': 4.0, 'close': -3.0}","{'extreme': 0.0, 'extension': 4.0, 'close': 1.0}","{'extreme': 0.0, 'extension': 4.0, 'close': 1.0}","{'extreme': 0.0, 'extension': 4.0, 'close': -3.0}","{'extreme': 0.0, 'extension': 4.0, 'close': 1.0}",...,"{'extreme': 0.0, 'extension': 2.0, 'close': -1.0}","{'extreme': 0.0, 'extension': 4.0, 'close': 3.0}","{'extreme': 1.0, 'extension': 4.0, 'close': 1.0}","{'extreme': 0.0, 'extension': 4.0, 'close': 1.0}","{'extreme': 0.0, 'extension': 4.0, 'close': 1.0}","{'extreme': 0.0, 'extension': 4.0, 'close': 1.0}","{'extreme': 0.0, 'extension': 4.0, 'close': -1.0}","{'extreme': 0.0, 'extension': 2.0, 'close': -1.0}","{'extreme': 0.0, 'extension': 4.0, 'close': -1.0}","{'extreme': 3.0, 'extension': 4.0, 'close': 3.0}"
